<a href="https://colab.research.google.com/github/sgevatschnaider/blockchain-finanzas-descentralizadas/blob/main/unidades/u05-indicadores-trading/python/Estrategias_RedesNeuronales_Lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# -*- coding: utf-8 -*-
# ===============================================================
# LSTM para predicción de cierre diario de Bitcoin (BTC-USD)
# Visualmente atractivo y didáctico para clase - Google Colab
# Autor: Profesor Sergio Gevatschnaider
# ===============================================================

# 1) Instalar/Importar librerías (Colab suele traer varias preinstaladas)
!pip -q install yfinance plotly==5.* scikit-learn==1.* --upgrade

import os, sys, warnings, math, random
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import yfinance as yf

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 2) Reproducibilidad
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

# 3) Configuración general
TICKER = "BTC-USD"
START  = "2016-01-01"
END    = None            # Hasta hoy
TEST_SIZE = 0.2          # 20% últimos datos para test
SEQ_LEN   = 60           # ventana (días) para las secuencias
EPOCHS    = 40
BATCH     = 32
LR        = 1e-3

# 4) Descarga de datos
df = yf.download(TICKER, start=START, end=END, progress=False)
assert not df.empty, "No se descargaron datos. Revisa conexión o símbolo."

df = df[['Open','High','Low','Close','Volume']].copy()
df.dropna(inplace=True)

# 5) Features simples + objetivo
# Objetivo: precio de Cierre (Close). Añadimos features: retornos y rango intradía.
df['Return']      = df['Close'].pct_change()
df['Volatility']  = (df['High'] - df['Low']) / df['Open']
df['LogVolume']   = np.log1p(df['Volume'])
df.dropna(inplace=True)

FEATURES = ['Close','Open','High','Low','Return','Volatility','LogVolume']
TARGET   = 'Close'

# 6) Split temporal (train/test por tiempo, sin barajar)
n_total = len(df)
n_test  = int(n_total * TEST_SIZE)
train_df = df.iloc[:-n_test].copy()
test_df  = df.iloc[-n_test:].copy()

# 7) Escalado MinMax por estabilidad del entrenamiento (fit SOLO con train)
scaler_x = MinMaxScaler(feature_range=(0,1))
scaler_y = MinMaxScaler(feature_range=(0,1))

X_train_raw = train_df[FEATURES].values
y_train_raw = train_df[[TARGET]].values

X_test_raw  = test_df[FEATURES].values
y_test_raw  = test_df[[TARGET]].values

X_train_scaled = scaler_x.fit_transform(X_train_raw)
X_test_scaled  = scaler_x.transform(X_test_raw)

y_train_scaled = scaler_y.fit_transform(y_train_raw)
y_test_scaled  = scaler_y.transform(y_test_raw)

# 8) Función para crear secuencias (X[t-SEQ_LEN:t], y[t])
def make_sequences(X, y, seq_len=60):
    Xs, ys = [], []
    for i in range(seq_len, len(X)):
        Xs.append(X[i-seq_len:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_train, y_train = make_sequences(X_train_scaled, y_train_scaled, SEQ_LEN)
X_test,  y_test  = make_sequences(X_test_scaled,  y_test_scaled,  SEQ_LEN)

print(f"Dimensiones -> X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Dimensiones -> X_test : {X_test.shape},  y_test : {y_test.shape}")

# 9) Modelo LSTM (sencillo, estable y didáctico)
def build_model(n_features, seq_len=60, lr=1e-3):
    model = keras.Sequential([
        layers.Input(shape=(seq_len, n_features)),
        layers.LSTM(64, return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(32),
        layers.Dropout(0.2),
        layers.Dense(16, activation="relu"),
        layers.Dense(1, activation="linear")
    ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss="mse")
    return model

model = build_model(n_features=X_train.shape[-1], seq_len=SEQ_LEN, lr=LR)
model.summary()

# 10) Callbacks: EarlyStopping + ReduceLROnPlateau para estabilidad
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-5, verbose=1)
]

# 11) Entrenamiento
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=EPOCHS,
    batch_size=BATCH,
    shuffle=False,   # MUY IMPORTANTE en series temporales
    callbacks=callbacks,
    verbose=1
)

# 12) Predicciones e inverse transform
y_pred_scaled = model.predict(X_test, verbose=0)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_true = scaler_y.inverse_transform(y_test)

# Alinear índices con la porción de test efectiva (por SEQ_LEN perdido)
test_index = test_df.index[SEQ_LEN:]

pred_df = pd.DataFrame({
    "Date": test_index,
    "y_true": y_true.flatten(),
    "y_pred": y_pred.flatten()
}).set_index("Date")

# 13) Métricas
rmse = math.sqrt(mean_squared_error(pred_df["y_true"], pred_df["y_pred"]))
mape = mean_absolute_percentage_error(pred_df["y_true"], pred_df["y_pred"]) * 100
print(f"RMSE: {rmse:,.2f}")
print(f"MAPE: {mape:,.2f}%")

# 14) Visualizaciones (Plotly: interactivo y prolijo para clase)

# 14.1) Precio histórico + anotaciones
fig_price = go.Figure()
fig_price.add_trace(go.Scatter(
    x=df.index, y=df['Close'], mode='lines',
    name='BTC-USD Close'
))
fig_price.update_layout(
    title=f'Precio histórico de {TICKER}',
    xaxis_title='Fecha', yaxis_title='Precio (USD)',
    template='plotly_white', height=450
)
fig_price.show()

# 14.2) Curvas de entrenamiento (loss)
fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(
    x=list(range(1,len(history.history['loss'])+1)),
    y=history.history['loss'], mode='lines', name='Train Loss'
))
fig_loss.add_trace(go.Scatter(
    x=list(range(1,len(history.history['val_loss'])+1)),
    y=history.history['val_loss'], mode='lines', name='Val Loss'
))
fig_loss.update_layout(
    title='Curvas de entrenamiento (MSE)',
    xaxis_title='Época', yaxis_title='Pérdida',
    template='plotly_white', height=400
)
fig_loss.show()

# 14.3) Real vs Predicho en Test
fig_pred = go.Figure()
fig_pred.add_trace(go.Scatter(
    x=pred_df.index, y=pred_df['y_true'], mode='lines', name='Real'
))
fig_pred.add_trace(go.Scatter(
    x=pred_df.index, y=pred_df['y_pred'], mode='lines', name='Predicción'
))
fig_pred.update_layout(
    title=f'Real vs. Predicho (Test) — RMSE: {rmse:,.2f} | MAPE: {mape:,.2f}%',
    xaxis_title='Fecha', yaxis_title='Precio (USD)',
    template='plotly_white', height=450
)
fig_pred.show()

# 14.4) Velas + predicción superpuesta (últimos ~180 días test)
tail_n = min(180, len(test_df))
tail_candles = test_df.tail(tail_n)

fig_c = go.Figure(data=[
    go.Candlestick(
        x=tail_candles.index,
        open=tail_candles['Open'],
        high=tail_candles['High'],
        low=tail_candles['Low'],
        close=tail_candles['Close'],
        name='Velas'
    )
])

# Agregar predicciones en el mismo rango temporal
pred_tail = pred_df.loc[tail_candles.index.intersection(pred_df.index)]
fig_c.add_trace(go.Scatter(
    x=pred_tail.index, y=pred_tail['y_pred'],
    mode='lines', name='Predicción (Close)'
))
fig_c.update_layout(
    title='Velas (BTC) + Predicción LSTM',
    xaxis_title='Fecha', yaxis_title='Precio (USD)',
    template='plotly_white', height=550, xaxis_rangeslider_visible=False
)
fig_c.show()

# 15) Mini backtest didáctico (señal naive: si y_pred[t] > y_true[t-1] => long 1 día)
# Nota: Solo para docencia; NO es consejo financiero.
bt = pred_df.copy()
bt['prev_close'] = bt['y_true'].shift(1)
bt['signal'] = (bt['y_pred'] > bt['prev_close']).astype(int)  # 1 si predicción > último cierre
bt['ret'] = bt['y_true'].pct_change().fillna(0)

# Rendimiento estrategia (estar long cuando signal==1)
bt['strategy_ret'] = bt['signal'].shift(1).fillna(0) * bt['ret']  # entrar al día siguiente
bt['cum_strategy'] = (1 + bt['strategy_ret']).cumprod()
bt['cum_buyhold']  = (1 + bt['ret']).cumprod()

fig_bt = go.Figure()
fig_bt.add_trace(go.Scatter(x=bt.index, y=bt['cum_buyhold'],  mode='lines', name='Buy & Hold'))
fig_bt.add_trace(go.Scatter(x=bt.index, y=bt['cum_strategy'], mode='lines', name='Estrategia LSTM (naive)'))
fig_bt.update_layout(
    title='Backtest didáctico: LSTM naive vs Buy & Hold',
    xaxis_title='Fecha', yaxis_title='Crecimiento acumulado (x)',
    template='plotly_white', height=450
)
fig_bt.show()

# 16) Cuadro resumen (métricas y parámetros) para mostrar en clase
summary = pd.DataFrame({
    'Ticker':[TICKER],
    'Inicio':[df.index.min().date()],
    'Fin':[df.index.max().date()],
    'Observaciones':[len(df)],
    'Ventana (días)':[SEQ_LEN],
    'Test (%)':[int(TEST_SIZE*100)],
    'RMSE':[round(rmse,2)],
    'MAPE (%)':[round(mape,2)],
    'Épocas reales':[len(history.history['loss'])],
    'Batch Size':[BATCH],
    'LR':[LR]
})
summary


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 72.3 MB/s eta 0:00:00
Dimensiones -> X_train: (2789, 60, 7), y_train: (2789, 1)
Dimensiones -> X_test : (652, 60, 7),  y_test : (652, 1)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 60, 64)         │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 60, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,393 (122.63 KB)

 Trainable params: 31,393 (122.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
79/79 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - loss: 0.0051 - val_loss: 0.0343 - learning_rate: 0.0010
Epoch 2/40
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - loss: 0.0080 - val_loss: 0.0520 - learning_rate: 0.0010
Epoch 3/40
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - loss: 0.0093 - val_loss: 0.0378 - learning_rate: 0.0010
Epoch 4/40
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0082
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - loss: 0.0084 - val_loss: 0.0444 - learning_rate: 0.0010
Epoch 5/40
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - loss: 0.0313 - val_loss: 0.0049 - learning_rate: 5.0000e-04
Epoch 6/40
79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - loss: 0.0695 - val_loss: 0.0151 - learning_rate: 5.0000e-04
Epoch 7/40
79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 0.0544 - val_loss: 0.0190 - learning_rate: 5.0000e-04
Epoch 8/40
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0498
Epoch 8: ReduceLROnPlateau r

,Ticker,Inicio,Fin,Observaciones,Ventana (días),Test (%),RMSE,MAPE (%),Épocas reales,Batch Size,LR
0,BTC-USD,2016-01-02,2025-10-02,3561,60,20,42767.11,41.64,19,32,0.001
